In [ ]:
import os
import random
import torch
import torchvision.transforms as transforms

from torchvision.io import read_image

import dill

In [ ]:
# Function to load the pre-trained model
def load_model(model_path,multi = True):
    if multi:
        with open(model_path, 'rb') as f:
            model = dill.load(f)
        return model
    else:
        model = torch.load(model_path)
        return model

In [ ]:
def preprocess_image(image_path):
    transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224 x 224
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize the images
])

    image = read_image(image_path).float()  # Load and convert the image to a float tensor
    image = image.repeat(3, 1, 1)  # Convert the image to 3 channels
    image = transform(image)
    image = image.unsqueeze(0)  # Add a batch dimension

    return image

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Function to get the output class
def get_output_class(model, image):
    with torch.no_grad():
        output = model(image.to(device))  # Move the image to the appropriate device
        _, predicted = torch.max(output.data, 1)
        predicted_class = predicted.item()

    # Map the predicted class index to the actual class name
    classes = ['Covid19', 'Pneumonia', 'Negative']  # Replace with your actual class names
    output_class = classes[predicted_class]

    return output_class

In [ ]:
# Load the pre-trained model
model_path = '/home/krishnatejaswis/Files/VSCode/LungScan/Solo Models/vgg19_model.pth'
model = load_model(model_path,multi=False)
modelName = 'vgg19_model'
model.eval()

In [ ]:
# Define the root directory containing class directories
root_dir = '/home/krishnatejaswis/Files/VSCode/LungScan/DataSetkaggle'

In [ ]:
# Initialize list to store validation image paths
validation_image_files = []

In [ ]:
# Iterate through each class directory
for class_name in os.listdir(root_dir):
    #print(class_name)
    class_dir = os.path.join(root_dir, class_name)
    if os.path.isdir(class_dir):
        # Get list of image files in the class directory
        class_images = [os.path.join(class_dir, f) for f in os.listdir(class_dir)]
        #print(class_images)
        # Shuffle the image files
        random.shuffle(class_images)
        print(len(class_images))
        # Select 30 images for validation from the shuffled list
        #validation_image_files.extend(class_images[:30])
        for i in range(len(class_images)):
            validation_image_files.append((class_images[i],f'{class_name}'))

In [ ]:
#print(len(validation_image_files))

In [ ]:
# Shuffle the selected images again
#random.shuffle(validation_image_files)
#print(validation_image_files)
outputTuples= []


In [ ]:

# Validate the model on the selected images
for image_file in validation_image_files:
    input_image = preprocess_image(image_file[0])
    output_class = get_output_class(model, input_image)
    #print(f'Predicted class for {image_file}: {output_class}')
    outputTuples.append((image_file,output_class))

In [ ]:
#print(len(outputTuples))

In [ ]:
counts = {'Covid19':0,'Pneumonia':0,'Negative':0}
for obj in outputTuples:
    if obj[1] == 'Covid19':
        counts['Covid19']+=1
    elif obj[1] == 'Pneumonia':
        counts['Pneumonia']+=1
    elif obj[1]=='Negative':
        counts['Negative']+=1
print(counts)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def get_output_class(model, image):
    with torch.no_grad():
        output = model(image.to(device))
        _, predicted = torch.max(output.data, 1)
        predicted_class = predicted.item()

    return predicted_class

In [ ]:
true_labels = []
pred_labels = []

In [ ]:
model.eval()
for image_file in validation_image_files:
    input_image = preprocess_image(image_file[0])
    true_label = ['Covid19', 'Pneumonia', 'Negetive'].index(image_file[1])
    true_labels.append(true_label)
    predicted_class = get_output_class(model, input_image)
    pred_labels.append(predicted_class)
    #print(f'Predicted class for {image_file[0]}: {["Covid19", "Pneumonia", "Negetive"][predicted_class]}')

In [ ]:
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='macro')
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')

In [ ]:
cm = confusion_matrix(true_labels, pred_labels)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig(f"./ValidationPlots/ConfusionMatrix{modelName}.png")
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.bar(['Covid19', 'Pneumonia', 'Negative'], [counts['Covid19'], counts['Pneumonia'], counts['Negative']])
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.savefig(f"./ValidationPlots/ClassDistribution{modelName}.png")
plt.show()

In [ ]:
# Print classification report
from sklearn.metrics import classification_report
print("\nClassification Report:")
class_names = ['Covid19', 'Pneumonia', 'Negative']
print(classification_report(true_labels, pred_labels, target_names=class_names))
print(f"F1 score ={f1:.4f}, Accuracy ={accuracy:.4f}, Precision = {precision:.4f}, recall = {recall:.4f} ")
# Specify the file path
output_file = '/home/krishnatejaswis/Files/VSCode/LungScan/ValidationPlots/classification_report.txt'

# Open the file in write mode
with open(output_file, 'a') as file:
    # Write the classification report into the file
    file.write(f"\n\nClassification Report for {modelName}:\n")
    file.write(classification_report(true_labels, pred_labels, target_names=class_names))
    file.write(f"F1 score ={f1:.4f}, Accuracy ={accuracy:.4f}, Precision = {precision:.4f}, recall = {recall:.4f} \n")

# Print a message to confirm that the file has been written
print(f"Classification report has been written to {output_file}")

